# Keras模型训练

### 构建(3种方式)-训练(model.fit())-验证（model.evaluate()）-预测(model.predict())-保存-加载
     - 使用样本加权（weight），和类别加权（不均衡样本）

### 回调函数
    - ModelCheckpoint
    - earlystopping
    - TensorBoard：模型日志
    - CSVLogger:输出到

In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-rc0


## 案例:
1.1 框架结构
- 构建模型（顺序模型、函数式模型、子类模型）
- 模型训练：model.fit()+权重+回调函数
- 模型验证：model.evaluate()
- 模型预测： model.predict()

In [2]:
def get_model():
    inputs = tf.keras.Input(shape=(32,), name='digits')
    x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
    x = tf.keras.layers.Dense(64, activation='relu', name='dense_2')(x)
    outputs = tf.keras.layers.Dense(10, name='predictions')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001), #优化器
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #损失函数
                  metrics=['accuracy'])
    return model

In [3]:
###构建数据集
#
import numpy as np
x_train = np.random.random((1000, 32))
y_train = np.random.randint(10, size=(1000, ))

x_val = np.random.random((200, 32))
y_val = np.random.randint(10, size=(200, ))

x_test = np.random.random((200, 32))
y_test = np.random.randint(10, size=(200, ))

#类别5：加权
# class_weight = {0: 1., 1: 1., 2: 1., 3: 1., 4: 1.,
#                 5: 2.,
#                 6: 1., 7: 1., 8: 1., 9: 1.}

# 上面与下面的效果等同
# Here's the same example using `sample_weight` instead:
sample_weight = np.ones(shape=(len(y_train),))

sample_weight[y_train == 5] = 2.
print('\nFit with sample weight')


Fit with sample weight


In [4]:
call_backs = [
#     tf.keras.callbacks.EarlyStopping(
#         # 当‘val_loss’不再下降时候停止训练 
#         monitor='val_loss',
#         # “不再下降”被定义为“减少不超过1e-2”
#         min_delta=1e-2,
#         # “不再改善”进一步定义为“至少2个epoch”
#         patience=2,
#         verbose=1),
    
#     tf.keras.callbacks.ModelCheckpoint(
#         filepath='mymodel_{epoch}',
#         # 模型保存路径
#         # 下面的两个参数意味着当且仅当`val_loss`分数提高时，我们才会覆盖当前检查点。
#         save_best_only=True,
#         monitor='val_loss',
#         #加入这个仅仅保存模型权重
#         save_weights_only=True,
#         verbose=1),
    
    
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_sparse_categorical_accuracy", 
                                             verbose=1, 
                                             mode='max', 
                                             factor=0.5, 
                                             patience=3)
]

In [5]:
model=get_model()
model.fit(x_train, y_train, 
          sample_weight=sample_weight,
          batch_size=32, 
          epochs=5, 
          validation_data=  (x_val, y_val) ,
         callbacks=call_backs)

# 自动切割一部分数据集作为
# model.fit(x_train, y_train, batch_size=64, validation_split=0.2, epochs=1)

Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 2.5339 - accuracy: 0.1070 - val_loss: 2.3624 - val_accuracy: 0.1050
Epoch 2/5
1000/1000 [==============================] - 0s 158us/sample - loss: 2.5007 - accuracy: 0.1090 - val_loss: 2.3668 - val_accuracy: 0.1050
Epoch 3/5
1000/1000 [==============================] - 0s 110us/sample - loss: 2.4888 - accuracy: 0.1090 - val_loss: 2.3600 - val_accuracy: 0.1050
Epoch 4/5
1000/1000 [==============================] - 0s 109us/sample - loss: 2.4811 - accuracy: 0.1090 - val_loss: 2.3660 - val_accuracy: 0.1050
Epoch 5/5
1000/1000 [==============================] - 0s 108us/sample - loss: 2.4647 - accuracy: 0.1100 - val_loss: 2.3681 - val_accuracy: 0.1050


In [6]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_test[:3])
print('predictions shape:', predictions.shape)


# Evaluate on test data
200/1 [========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

## 案例3、使用回调函数

Keras中的回调是在训练期间（在某个时期开始时，在批处理结束时，在某个时期结束时等）在不同时间点调用的对象，这些对象可用于实现以下行为：

在训练过程中的不同时间点进行验证（除了内置的按时间段验证）

定期或在超过特定精度阈值时对模型进行检查

当训练似乎停滞不前时，更改模型的学习率

当训练似乎停滞不前时，对顶层进行微调

在训练结束或超出特定性能阈值时发送电子邮件或即时消息通知
等等。
回调可以作为列表传递给model.fit：

### 3.1 EarlyStopping(早停)
- monitor: 被监测的数据。
- min_delta: 在被监测的数据中被认为是提升的最小变化， 例如，小于 min_delta 的绝对变化会被认为没有提升。
- patience: 没有进步的训练轮数，在这之后训练就会被停止。
- verbose: 详细信息模式。
- mode: {auto, min, max} 其中之一。 在 min 模式中， 当被监测的数据停止下降，训练就会停止；在 max 模式中，当被监测的数据停止上升，训练就会停止；在 auto 模式中，方向会自动从被监测的数据的名字中判断出来。

#### 许多内置的回调可用
- ModelCheckpoint：定期保存模型。
- EarlyStopping：当培训不再改善验证指标时，停止培训。
- TensorBoard：定期编写可在TensorBoard中可视化的模型日志（更多详细信息，请参见“可视化”部分）。
- CSVLogger：将损失和指标数据流式传输到CSV文件。
等等

### 3.2 checkpoint模型
在相对较大的数据集上训练模型时，至关重要的是要定期保存模型的checkpoint。

最简单的方法是使用ModelCheckpoint回调：

### 3.3、使用回调实现动态学习率调整
由于优化程序无法访问验证指标，因此无法使用这些计划对象来实现动态学习率计划（例如，当验证损失不再改善时降低学习率）。

但是，回调确实可以访问所有指标，包括验证指标！因此，可以通过使用回调来修改优化程序上的当前学习率，从而实现此模式。实际上，它是作为ReduceLROnPlateau回调内置的。

ReduceLROnPlateau参数

- monitor: 被监测的指标。
- factor: 学习速率被降低的因数。新的学习速率 = 学习速率 * 因数
- patience: 没有进步的训练轮数，在这之后训练速率会被降低。
- verbose: 整数。0：安静，1：更新信息。
- mode: {auto, min, max} 其中之一。如果是 min 模式，学习速率会被降低如果被监测的数据已经停止下降； 在 max 模式，学习塑料会被降低如果被监测的数据已经停止上升； 在 auto 模式，方向会被从被监测的数据中自动推断出来。
- min_delta: 衡量新的最佳阈值，仅关注重大变化。
- cooldown: 在学习速率被降低之后，重新恢复正常操作之前等待的训练轮数量。
- min_lr: 学习速率的下边界。

模型保存有两种需求，一种是简单的保存、加载+预测，还有一种是保存，部署+加载预测。
如果是第一种需求，使用下面的代码就可以了
## 模型保存+加载+预测
最终保存的格式是pb


In [7]:
# Export the model to a SavedModel
model.save('keras_model_tf_version', save_format='tf')

# Recreate the exact same model
new_model = tf.keras.models.load_model('keras_model_tf_version')
new_model.predict(x_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: keras_model_tf_version\assets


array([[-0.1415214 , -0.12553616, -0.2189347 , ..., -0.05956711,
        -0.31231803, -0.0866922 ],
       [ 0.00869526, -0.260038  , -0.12676363, ..., -0.09518217,
        -0.17964485, -0.02762674],
       [-0.04753163, -0.12072425, -0.19626844, ...,  0.17215061,
        -0.15453073,  0.12552463],
       ...,
       [ 0.04350837, -0.21960051, -0.04866163, ..., -0.07364215,
        -0.09294329,  0.16136262],
       [ 0.04430031, -0.04118293, -0.15946655, ...,  0.04697893,
        -0.18407892,  0.15345633],
       [-0.2637505 , -0.10963977, -0.22710171, ..., -0.27079043,
        -0.22774127, -0.08513794]], dtype=float32)

## 模型保存+部署+加载预测

In [11]:
tf.saved_model.save(model,'keras_saved_model_version')

INFO:tensorflow:Assets written to: keras_saved_model_version\assets


In [13]:
restored_saved_model = tf.saved_model.load('keras_saved_model_version')

In [16]:
!saved_model_cli show --dir keras_saved_model_version --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['digits'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32)
        name: serving_default_digits:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['predictions'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


In [17]:
f = restored_saved_model.signatures["serving_default"]

In [18]:
f(digits = tf.constant(x_test.tolist()) )

{'predictions': <tf.Tensor: id=5298, shape=(200, 10), dtype=float32, numpy=
 array([[-0.1415214 , -0.12553616, -0.2189347 , ..., -0.05956711,
         -0.31231803, -0.0866922 ],
        [ 0.00869526, -0.260038  , -0.12676363, ..., -0.09518217,
         -0.17964485, -0.02762674],
        [-0.04753163, -0.12072425, -0.19626844, ...,  0.17215061,
         -0.15453073,  0.12552463],
        ...,
        [ 0.04350837, -0.21960051, -0.04866163, ..., -0.07364215,
         -0.09294329,  0.16136262],
        [ 0.04430031, -0.04118293, -0.15946655, ...,  0.04697893,
         -0.18407892,  0.15345633],
        [-0.2637505 , -0.10963977, -0.22710171, ..., -0.27079043,
         -0.22774127, -0.08513794]], dtype=float32)>}